<a href="https://colab.research.google.com/github/somesh-awasthi/NLP-PROJECT/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install ctransformers==0.2.27 sentence-transformers==2.5.1 chromadb==0.4.24 langchain==0.1.11 PyPDF

#linking my drive with my notebook & giving the data path
from google.colab import drive
drive.mount("/content/drive")
path="/content/drive/MyDrive/Colab Notebooks/data"

import chromadb
import textwrap
import traceback
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
# from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [2]:
#loading data
def load_data(path):
    loader = DirectoryLoader( path, glob="Medical_book.pdf", loader_cls=PyPDFLoader )
    documents = loader.load()
    return documents

doc=load_data(path)

In [3]:
#Create text chunks
def text_split(data):
    splitted_text = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20, length_function=len, add_start_index=True )
    chunk = splitted_text.split_documents(data)
    return chunk

chunk = text_split(doc)
print(f"len of doc: {len(doc)} length of my chunk:{len(chunk)}" )

len of doc: 637 length of my chunk:7020


In [4]:
# how our chunk looks like
prints=chunk[10]
print(prints.page_content)
print(prints.metadata)

Deirdre S. Blanchfield, associate editor — 2nd ed.
p. cm.
Includes bibliographical references and index.
Contents: V ol. 1. A-B — v. 2. C-F — v. 3.G-M — v. 4. N-S — v. 5. T-Z.
ISBN 0-7876-5489-2 (set: hardcover) — ISBN 0-7876-5490-6
(vol. 1) — ISBN 0-7876-5491-4 (vol. 2) — ISBN 0-7876-5492-2(vol. 3) — ISBN 0-7876-5493-0 (vol. 4) — ISBN 0-7876-5494-9(vol. 5)
1. Internal medicine—Encyclopedias. I. Longe, Jacqueline L. 
II. Blanchfield, Deirdre S. III. Gale Research Company.
RC41.G35 2001
{'source': '/content/drive/MyDrive/Colab Notebooks/data/Medical_book.pdf', 'page': 3, 'start_index': 3284}


In [5]:
#embedding model
def embedding_model():
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding
embedding = embedding_model()
embedding

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
# Creating vector db to store Embeddings
persist_directory = "vectordb"
vectordb = Chroma.from_documents(documents=chunk, embedding=embedding, persist_directory=persist_directory)

In [7]:
vectordb.persist()
vectordb=None

In [8]:
# loading the db
vectordb=Chroma(embedding_function=embedding, persist_directory=persist_directory)
# retriving the query
retriever=vectordb.as_retriever()

In [9]:
#If we already have an index we can load it like this
r_docs=retriever.get_relevant_documents("What are Allergies")
len(r_docs)
r_docs[0]

Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={'page': 130, 'source': '/content/drive/MyDrive/Colab Notebooks/data/Medical_book.pdf', 'start_index': 2181})

In [10]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [11]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [12]:
llm=CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.ggmlv3.q2_K.bin:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

In [13]:
qa = RetrievalQA.from_chain_type( llm=llm, chain_type="stuff", retriever=vectordb.as_retriever(search_kwargs={'k': 2}), return_source_documents=True, chain_type_kwargs=chain_type_kwargs )

In [25]:
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text


def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [14]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])
# process_llm_response(result)

Input Prompt:what is brain tumor


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  Brain tumor refers to abnormal growths in the brain that can be benign or malignant. Benign brain tumors are non-cancerous and do not metastasize beyond the part of the brain where they originate. Malignant brain tumors, on the other hand, are composed of cancer cells and have the potential to spread beyond the brain.
